In [1]:
# !/usr/bin/env python
# -*- coding: utf-8 -*-

import requests
import re
import os
import pandas as pd
from time import gmtime, strftime
from models.RSS_classifier import classify, classify2

In [2]:
def get_articles_and_xml(url):
    page = requests.get(url)

    pattern = '<item rdf:about=.*?</item>'
    article_list = re.findall(pattern, page.text, flags=(re.MULTILINE | re.DOTALL))
    return article_list, page.text

In [3]:
def extract_item_info(xml_string):
    item_dict = {}
    item_dict['url'] = re.findall('http://arxiv.org/abs/\d{1,8}\.\d{1,8}', xml_string, flags=(re.MULTILINE | re.DOTALL))[0]
    item_dict['title'] = (re.findall('<title>(.*?)[(]arXiv:', xml_string, flags=(re.MULTILINE | re.DOTALL))[0]).strip()
    item_dict['abstract'] = re.findall('<description rdf:.*?>(.*?)</description>', xml_string, flags=(re.MULTILINE | re.DOTALL))[0]
    item_dict['authors'] = re.findall('<dc:creator>(.*?)</dc:creator>', xml_string, flags=(re.MULTILINE | re.DOTALL))[0]
    item_dict['authors'] = re.findall('\&quot;\&gt;(.*?)\&lt;/a\&gt;', item_dict['authors'], flags=(re.MULTILINE | re.DOTALL))
    item_dict['authors'] = ', '.join(item_dict['authors'])
    item_dict['full_text'] = xml_string
    return item_dict

In [4]:
def get_feed_parts(xml_string):
    beginning = re.findall('.*?<rdf:Seq>\n', xml_string, flags=(re.MULTILINE | re.DOTALL))[0]
    middle = re.findall('</rdf:Seq>.*?</image>', xml_string, flags=(re.MULTILINE | re.DOTALL))[0]
    end = '</rdf:RDF>'
    
    beginning_good = re.sub('<title>(.*?)</title>', '<title>Good papers from arXiv</title>', beginning, flags=(re.MULTILINE | re.DOTALL))
    beginning_bad = re.sub('<title>(.*?)</title>', '<title>Boring papers from arXiv</title>', beginning, flags=(re.MULTILINE | re.DOTALL))
    
    beginning_good = re.sub('<description(.*?)>(.*?)</description>', '<description\g<1>>The Good Papers on ML, AI and Statistics</description>', beginning_good, flags=(re.MULTILINE | re.DOTALL))
    beginning_bad = re.sub('<description(.*?)>(.*?)</description>', '<description\g<1>>The Bad Papers on ML, AI and Statistics</description>', beginning_bad, flags=(re.MULTILINE | re.DOTALL))
    
    return beginning_good, beginning_bad, middle, end

In [5]:
def build_feeds(xml_string, articles_info_classified_pd):
    beginning_good, beginning_bad, middle, end = get_feed_parts(xml_string)
    
    
    items_good = []
    items_bad = []
    for i in range(len(articles_info_classified_pd)):
        item = '<rdf:li rdf:resource="' + articles_info_classified_pd['url'].iloc[i] + '"/>'
        if articles_info_classified_pd['class'].iloc[i] == True:
            items_good.append(item)
        else:
            items_bad.append(item)
            
    items_good = '\n'.join(items_good)
    items_bad = '\n'.join(items_bad)
    
    abstracts_good = []
    abstracts_bad = []
    for i in range(len(articles_info_classified_pd)):
        item = articles_info_classified_pd['full_text'].iloc[i]
        if articles_info_classified_pd['class'].iloc[i] == True:
            abstracts_good.append(item)
        else:
            abstracts_bad.append(item)
            
    abstracts_good = '\n'.join(abstracts_good)
    abstracts_bad = '\n'.join(abstracts_bad)
    
    good_text = beginning_good + items_good + middle + abstracts_good + end
    bad_text = beginning_bad + items_bad + middle + abstracts_bad + end
    
    return good_text, bad_text

In [6]:
urls = [
    'http://arxiv.org/rss/cs.NE',
    'http://arxiv.org/rss/cs.AI',
    'http://arxiv.org/rss/stat.ML'
]
good_feed_name = 'papers_good.xml'
bad_feed_name = 'papers_bad.xml'

articles_info_classified_list = []
for url in urls:
    article_list, xml_string = get_articles_and_xml(url)

    articles_info_list = []
    for article in article_list:
        info = extract_item_info(article)
        articles_info_list.append(info)

    articles_info_pd = pd.DataFrame(articles_info_list)
    articles_info_classified_list.append(classify(articles_info_pd))
    #articles_info_classified_pd = classify(articles_info_pd)

articles_info_classified_pd = pd.concat([articles_info_classified_list[0], 
                                              articles_info_classified_list[1],
                                              articles_info_classified_list[2]])

articles_info_classified_pd = articles_info_classified_pd.drop_duplicates(subset=['url'])

{'svc': Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])}
{'svc': Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])}
{'svc': Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-

In [7]:
articles_info_classified_pd

,abstract,authors,full_text,title,url,class
0,&lt;p&gt;Fuzzy clustering has become a widely ...,"Saptarshi Sengupta, Sanchita Basak, Richard Al...","<item rdf:about=""http://arxiv.org/abs/1712.055...",Data Clustering using a Hybrid of Fuzzy C-Mean...,http://arxiv.org/abs/1712.05512,False
1,&lt;p&gt;Most of the weights in a Lightweight ...,Altaf H. Khan,"<item rdf:about=""http://arxiv.org/abs/1712.056...",Lightweight Neural Networks.,http://arxiv.org/abs/1712.05695,True
2,&lt;p&gt;Representing the semantic relations t...,"Huda Hakami, Danushka Bollegala, Hayashi Kohei","<item rdf:about=""http://arxiv.org/abs/1709.066...",Why PairDiff works? -- A Mathematical Analysis...,http://arxiv.org/abs/1709.06673,True
3,&lt;p&gt;Neuromorphic hardware tends to pose l...,"Guillaume Bellec, David Kappel, Wolfgang Maass...","<item rdf:about=""http://arxiv.org/abs/1711.051...",Deep Rewiring: Training very sparse deep netwo...,http://arxiv.org/abs/1711.05136,True
0,&lt;p&gt;Aspect-based sentiment analysis (ABSA...,"Yi Tay, Anh Tuan Luu, Siu Cheung Hui","<item rdf:about=""http://arxiv.org/abs/1712.054...",Learning to Attend via Word-Aspect Associative...,http://arxiv.org/abs/1712.05403,True
1,&lt;p&gt;We introduce The House Of inteRaction...,"Eric Kolve, Roozbeh Mottaghi, Daniel Gordon, Y...","<item rdf:about=""http://arxiv.org/abs/1712.054...",AI2-THOR: An Interactive 3D Environment for Vi...,http://arxiv.org/abs/1712.05474,True
2,&lt;p&gt;When presented with an unknown robot ...,"Ashwin Khadke, Manuela Veloso","<item rdf:about=""http://arxiv.org/abs/1712.054...",What Can This Robot Do? Learning from Appearan...,http://arxiv.org/abs/1712.05497,False
3,&lt;p&gt;Inverse Reinforcement Learning (IRL) ...,"Siddharthan Rajasekaran, Jinwei Zhang, Jie Fu","<item rdf:about=""http://arxiv.org/abs/1712.055...",Inverse Reinforce Learning with Nonparametric ...,http://arxiv.org/abs/1712.05514,False
4,"&lt;p&gt;In this work, we propose a goal-drive...","Jin-Hwa Kim, Devi Parikh, Dhruv Batra, Byoung-...","<item rdf:about=""http://arxiv.org/abs/1712.055...",CoDraw: Visual Dialog for Collaborative Drawing.,http://arxiv.org/abs/1712.05558,True
6,&lt;p&gt;We suggest a model of a multi-agent s...,"V.I. Yukalov, E.P. Yukalova, D. Sornette","<item rdf:about=""http://arxiv.org/abs/1712.057...",Information Processing by Networks of Quantum ...,http://arxiv.org/abs/1712.05734,False


In [ ]:
good_text, bad_text = build_feeds(xml_string, articles_info_classified_pd)

with open(good_feed_name, 'w') as good:
    good.write(good_text)
    
with open(bad_feed_name, 'w') as bad:
    bad.write(bad_text)

In [ ]:
import git
repo = git.Repo( '../rss-classifier' )
print(repo.git.add( '.' ))

timestring = strftime("%Y%m%d_%H%M%S", gmtime())
message = 'Update at ' + timestring

print(repo.git.commit( m=message ))
print(repo.git.push())
print(repo.git.status())